In [2]:
import os 
import zipfile
import pandas as pd
import numpy as np

import bbg_symbology as bbs
from bbg_loader_core import *

In [3]:
def merge_dict(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

class ExpiredAliasService:
    def __init__(self, src=None, dbfldr=None):
        
        self._src = src
        self._dbfldr = dbfldr
        
        if src is not None:
            self.expired = list(set(pd.read_csv(src, header=None)[1]))
            
            if self._dbfldr is not None:
                dblist = self.get_db_list()
                self.expired = [e for e in self.expired if e in dblist]
        else:
            self.expired = []
    
    def get_db_list(self):
        extension = '.pickle'
        return [f.replace(extension, '') for f in os.listdir(self._dbfldr) if f.endswith(extension)]
    
    def __contains__(self, key):
        return key in self.expired
    
    def __repr__(self):
        return '<{}: {}>'.format(self.__class__, str(self.expired))
    
    def add_expired(self, aliases):
        if isinstance(aliases, list):
            self.expired = self.expired + aliases
        else:
            self.expired.extend([aliases])
        
    def save(self, fname=None):
        if fname is None:
            fname = self._src
            
        if fname is None:
            raise Error('No Filename Specified!')
            
        pd.Series(self.expired).to_csv(fname)
    

In [4]:
def generate_futures_list(out_path):
    
    # init - flds & default params
    ts_flds =   ['px_last', 'open_int', 'volume']
    meta_flds = ['LAST_TRADEABLE_DT']

    default_params = {'local_path':  out_path, 'ts_flds': ts_flds, 'meta_flds': meta_flds}
    
    # get all unique roots (alias roots)
    roots = list(FutAliasRef._fut_ref.Alias)
    
    # store outputs
    fut_list = {}
    
    # loop through roots
    for root in roots:
        
        # get all bbg, alias tckrs
        bbg_tckrs  = FutChainRef.get_futures_chain(root)
        alias_tckrs = [FutAliasRef.bbg_to_alias_tckr(tckr) for tckr in bbg_tckrs]
        tckr_list = zip(bbg_tckrs, alias_tckrs)
        
        # loop through ticker combos... and return object
        for bbg, alias in tckr_list:
            if alias not in ExpiredFutures:
                p = merge_dict({'bb_tckr': bbg, 'alias': alias}, default_params)
                fut_list[alias] = BbgSecurity(**p)
                
    return fut_list


def generate_bbg_list(src_file, out_path, ts_flds=['px_last'], meta_flds=['NAME','LONG_COMP_NAME']):
    tckrList = pd.read_csv(src_file)
    default_params = {'local_path': out_path, 'ts_flds': ts_flds, 'meta_flds': meta_flds}

    secList = {}    
    for (i, (bbg,alias)) in tckrList.iterrows():
        d = merge_dict({'bb_tckr': bbg, 'alias': alias}, default_params)
        secList[bbg] = BbgSecurity(**d)

    return secList


def generate_generic_fut_list(out_path, ts_flds=['px_last'], meta_flds=['NAME','LONG_COMP_NAME']):

    def gen_obj(broot, rmtd, yk, ng, alias_root, alias_suff, dparams):
        bbg_tckr   = broot.upper()+ str(ng) + ' ' + rmtd + ' ' + yk
        alias_tckr = alias_root + str(ng) + '.' + alias_suff

        d = merge_dict({'bb_tckr': bbg_tckr, 'alias': alias_tckr}, dparams)
        return alias_tckr, BbgSecurity(**d)
    
    ts_flds =   ['px_last', 'open_int', 'volume']
    meta_flds = ['NAME', 'PX_POS_MULT_FACTOR', 'EXCH_CODE', 'FUT_EXCH_NAME_LONG', 
                 'CRNCY', 'FUT_TICK_SIZE', 'FUT_GEN_MONTH']

    default_params_ridx = {'local_path': out_path, 'ts_flds': ts_flds, 'meta_flds': meta_flds}
    default_params_px   = {'local_path': out_path, 'ts_flds': ts_flds[:1], 'meta_flds': meta_flds[:1]}
    
    
    n = 1
    secList = {}    
    for idx, (root, num_gen, yk, alias) in FutAliasRef._fut_ref.iterrows():
        # Return Index
        atckr, bbobj = gen_obj(root, 'B:00_0_D', yk, 1, alias, 'bbd', default_params_ridx)
        secList[atckr] = bbobj
        
        # Price 
        atckr, bbobj = gen_obj(root, 'B:00_0_N', yk, 1, alias, 'bbd.px', default_params_px)
        secList[atckr] = bbobj
        
        # Return Index
        atckr, bbobj = gen_obj(root, 'B:00_0_D', yk, num_gen, alias, 'bbd', default_params_px)
        secList[atckr] = bbobj
        
        # Price
        atckr, bbobj = gen_obj(root, 'B:00_0_N', yk, num_gen, alias, 'bbd.px', default_params_px)
        secList[atckr] = bbobj

    return secList

def update_bbg_securities(bbglist):
    expired = []
    error_list = []
    print('--------------UPDATING {}----------------'.format(db))
    for i, (alias, bbgObj) in enumerate(bbglist.items()):
        print('\n(' + str(i) + ') ' + str(bbgObj))

        try:
            bbgObj.update()
            if bbgObj.is_expired:
                expired.append(bbgObj.alias)
        except:
            try: 
                print('...Error with {}, attempting to load from scratch'.format(bbgObj.bb_tckr))
                bbgObj.load_from_scratch()
                bbgObj.save()
                print('...success!')

                if bbgObj.is_expired:
                    expired.append(bbgObj.alias)
            except:
                print('FATAL ERROR WITH {}'.format(bbgObj.bb_tckr))
                error_list.extend(bbgObj)
    
    return expired, error_list

In [14]:
def alias_to_date(alias):
    dtinfo = alias.split('.')[1]
    month_str = dtinfo[0]
    yr_str = dtinfo[1:]
    
    MONTH_REF = 'FGHJKMNQUVXZ'
    month =  MONTH_REF.find(month_str) + 1
    
    yr =  int(yr_str)
    if yr > 50:
        yr = yr + 1900
    else:
        yr = yr + 2000

    return pd.datetime(yr, month, 1)

def get_sec_health(bbg):
    #bbg.load_local_data()
    health = bbg.ts.count()
    
    if 'LAST_TRADEABLE_DT' in bbg.meta:
        health['exp_error'] = np.abs((bbg.meta.LAST_TRADEABLE_DT - alias_to_date(bbg.alias)).days)
    
    try: 
        health['first_dt'] = (bbg.ts.first_valid_index()).strftime('%Y-%m-%d')
    except: 
        health['first_dt'] = None
        
    try: 
        health['last_dt'] = (bbg.ts.last_valid_index()).strftime('%Y-%m-%d')
    except:
        health['last_dt'] = None
        
    health.name = bbg.alias
    return health

def filter_health(df):

    err = pd.DataFrame()
    
    ts_flds = ['px_last', 'open_int', 'volume']

    for fld in ts_flds:
        if fld in df:
            err[fld] = df.px_last==0

    if 'exp_error' in df:
        err['exp'] = df.exp_error > 50

    return df[err.any(axis=1)]

# TODO THIS FUNCTION NEEDS WORK
def export_db_health(dbname, dbfldr='../_bbgDB/'):
        DIR = dbfldr + dbname + '/'

        flist = [f for f in os.listdir(DIR) if f.endswith('.pickle')]
        health_list = []
        
        for f in flist:
            print(f)
            bbg = BbgSecurity.from_file(DIR+f)
            health_list.append(get_sec_health(bbg))

        return health_list
    
        healthdf = pd.DataFrame(health_list)
        healthdf.update({dbname:pd.DataFrame(health_list)})
        filter_health(healthdf).to_excel('../_bbgDB/health_' + dbname + '.xlsx')
        

def zip_folder(fldr, zip_dest_fldr, zip_filename):
    zf = zipfile.ZipFile(zipfldr + zipfilename, 'w', zipfile.ZIP_DEFLATED)
    for dirname, subdirs, files in os.walk(fldr):    
        zf.write(dirname)
        for filename in files:
            zf.write(os.path.join(dirname, filename))
    zf.close()


In [6]:
INPUT_PATH = '../_in/'

FUTURES_MONTHS = {m:i+1 for i,m in enumerate('FGHJKMNQUVXZ')}

FutChainRef    = bbs.FuturesChainReference(INPUT_PATH + 'futures_historical_chain.csv')
FutAliasRef    = bbs.FuturesAliasService(INPUT_PATH + 'fut_roots.csv')
BbgTckrService = bbs.BloombergTckrService()
ExpiredFutures = ExpiredAliasService('../_in/expired_aliases.csv', '../_bbgDB/Futures/')


In [7]:
BBG_UPDATE_LIST = {'CoT':           generate_bbg_list(src_file=INPUT_PATH + 'CoT.csv',           out_path='../_bbgDB/CoT/'),
                   'Index':         generate_bbg_list(src_file=INPUT_PATH + 'Index.csv',         out_path='../_bbgDB/Index/'),
                   'FX':            generate_bbg_list(src_file=INPUT_PATH + 'FX.csv',            out_path='../_bbgDB/FX/'), 
                   'InterestRates': generate_bbg_list(src_file=INPUT_PATH + 'InterestRates.csv', out_path='../_bbgDB/InterestRates/'),
                   'Futures_gen':   generate_generic_fut_list(                                   out_path='../_bbgDB/Futures_gen/'),
                   'Futures':       generate_futures_list(                                       out_path='../_bbgDB/Futures/')}

# print # securities in each
for name, db in BBG_UPDATE_LIST.items():
    print(name, len(db))


('Index', 4)
('CoT', 1084)
('FX', 68)
('Futures', 2086)
('Futures_gen', 230)
('InterestRates', 46)


In [8]:
tckrs = list(BBG_UPDATE_LIST['Futures'].keys())
tckrs_ = [tckr for tckr in tckrs if tckr.split('.')[0]=='ng']

In [9]:
tckr = 'gc.N84'
bb = BbgSecurity.from_file('../_bbgDB/Futures/' + tckr + '.pickle')
bb.is_expired

True

In [10]:
expired = {}
error_list = {}

for db, bbglist in BBG_UPDATE_LIST.items():
    expired[db], error_list[db] = update_bbg_securities(bbglist)

--------------UPDATING Index----------------

(0) <BBG SECURITY: tckr=SPGSCIP Index, alias=gsci.er, local_file=../_bbgDB/Index/>
...updating security SPGSCIP Index
...loading local file <../_bbgDB/Index/gsci.er.pickle> for security=SPGSCIP Index
...updating timeseries for SPGSCIP Index --> loading data from 09/23/2019 to 10/01/2019
...saving SPGSCIP Index to local file <../_bbgDB/Index/gsci.er.pickle>

(1) <BBG SECURITY: tckr=SPGSCITR Index, alias=gsci.tr, local_file=../_bbgDB/Index/>
...updating security SPGSCITR Index
...loading local file <../_bbgDB/Index/gsci.tr.pickle> for security=SPGSCITR Index
...updating timeseries for SPGSCITR Index --> loading data from 09/23/2019 to 10/01/2019
...saving SPGSCITR Index to local file <../_bbgDB/Index/gsci.tr.pickle>

(2) <BBG SECURITY: tckr=NEIXSTTI Index, alias=sgstti, local_file=../_bbgDB/Index/>
...updating security NEIXSTTI Index
...loading local file <../_bbgDB/Index/sgstti.pickle> for security=NEIXSTTI Index
...updating timeseries for N

C:\Anaconda\envs\python2\lib\site-packages\tia\bbg\v3api.py:409: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  frame = frame.reindex_axis(self.request.fields, axis=1)


...saving CMEOHNCP Index to local file <../_bbgDB/CoT/lh.nc.fo.sprd.pickle>

(1) <BBG SECURITY: tckr=CSCOCNCS Index, alias=cc.nc.fo.short, local_file=../_bbgDB/CoT/>
...updating security CSCOCNCS Index
...loading local file <../_bbgDB/CoT/cc.nc.fo.short.pickle> for security=CSCOCNCS Index
...could not read file: ../_bbgDB/CoT/cc.nc.fo.short.pickle
...loading metadata for CSCOCNCS Index
...loading timeseries for CSCOCNCS Index --> loading data from 1/1/1960 to 10/01/2019
...saving CSCOCNCS Index to local file <../_bbgDB/CoT/cc.nc.fo.short.pickle>

(2) <BBG SECURITY: tckr=CFCDLSWS Index, alias=sb.sd.fo.short, local_file=../_bbgDB/CoT/>
...updating security CFCDLSWS Index
...loading local file <../_bbgDB/CoT/sb.sd.fo.short.pickle> for security=CFCDLSWS Index
...could not read file: ../_bbgDB/CoT/sb.sd.fo.short.pickle
...loading metadata for CFCDLSWS Index
...loading timeseries for CFCDLSWS Index --> loading data from 1/1/1960 to 10/01/2019
...saving CFCDLSWS Index to local file <../_bbgDB

...saving CMEOHNCL Index to local file <../_bbgDB/CoT/lh.nc.fo.long.pickle>

(19) <BBG SECURITY: tckr=CSC1SNCP Index, alias=sb.nc.f.sprd, local_file=../_bbgDB/CoT/>
...updating security CSC1SNCP Index
...loading local file <../_bbgDB/CoT/sb.nc.f.sprd.pickle> for security=CSC1SNCP Index
...could not read file: ../_bbgDB/CoT/sb.nc.f.sprd.pickle
...loading metadata for CSC1SNCP Index
...loading timeseries for CSC1SNCP Index --> loading data from 1/1/1960 to 10/01/2019
...saving CSC1SNCP Index to local file <../_bbgDB/CoT/sb.nc.f.sprd.pickle>

(20) <BBG SECURITY: tckr=CME1FNCL Index, alias=fc.nc.f.long, local_file=../_bbgDB/CoT/>
...updating security CME1FNCL Index
...loading local file <../_bbgDB/CoT/fc.nc.f.long.pickle> for security=CME1FNCL Index
...could not read file: ../_bbgDB/CoT/fc.nc.f.long.pickle
...loading metadata for CME1FNCL Index
...loading timeseries for CME1FNCL Index --> loading data from 1/1/1960 to 10/01/2019
...saving CME1FNCL Index to local file <../_bbgDB/CoT/fc.nc.f

...saving CFFDAMMD Index to local file <../_bbgDB/CoT/w.mm.f.sprd.pickle>

(37) <BBG SECURITY: tckr=CFCDGSWL Index, alias=s.sd.fo.long, local_file=../_bbgDB/CoT/>
...updating security CFCDGSWL Index
...loading local file <../_bbgDB/CoT/s.sd.fo.long.pickle> for security=CFCDGSWL Index
...could not read file: ../_bbgDB/CoT/s.sd.fo.long.pickle
...loading metadata for CFCDGSWL Index
...loading timeseries for CFCDGSWL Index --> loading data from 1/1/1960 to 10/01/2019
...saving CFCDGSWL Index to local file <../_bbgDB/CoT/s.sd.fo.long.pickle>

(38) <BBG SECURITY: tckr=NYC1CCOL Index, alias=ct.cm.f.long, local_file=../_bbgDB/CoT/>
...updating security NYC1CCOL Index
...loading local file <../_bbgDB/CoT/ct.cm.f.long.pickle> for security=NYC1CCOL Index
...could not read file: ../_bbgDB/CoT/ct.cm.f.long.pickle
...loading metadata for NYC1CCOL Index
...loading timeseries for NYC1CCOL Index --> loading data from 1/1/1960 to 10/01/2019
...saving NYC1CCOL Index to local file <../_bbgDB/CoT/ct.cm.f.l

...

...saving BO1 B:00_0_D Comdty to local file <../_bbgDB/Futures_gen/bo1.bbd.pickle>

(189) <BBG SECURITY: tckr=PA6 B:00_0_D Comdty, alias=pa6.bbd, local_file=../_bbgDB/Futures_gen/>
...updating security PA6 B:00_0_D Comdty
...loading local file <../_bbgDB/Futures_gen/pa6.bbd.pickle> for security=PA6 B:00_0_D Comdty
...loading metadata for PA6 B:00_0_D Comdty
...loading timeseries for PA6 B:00_0_D Comdty --> loading data from 1/1/1960 to 10/01/2019
...saving PA6 B:00_0_D Comdty to local file <../_bbgDB/Futures_gen/pa6.bbd.pickle>

(190) <BBG SECURITY: tckr=LH1 B:00_0_D Comdty, alias=lh1.bbd, local_file=../_bbgDB/Futures_gen/>
...updating security LH1 B:00_0_D Comdty
...loading local file <../_bbgDB/Futures_gen/lh1.bbd.pickle> for security=LH1 B:00_0_D Comdty
...updating timeseries for LH1 B:00_0_D Comdty --> loading data from 09/24/2019 to 10/01/2019
...saving LH1 B:00_0_D Comdty to local file <../_bbgDB/Futures_gen/lh1.bbd.pickle>

(191) <BBG SECURITY: tckr=QS13 B:00_0_D Comdty, alias=q

...saving CL1 B:00_0_D Comdty to local file <../_bbgDB/Futures_gen/cl1.bbd.pickle>

(209) <BBG SECURITY: tckr=BA1 B:00_0_N Comdty, alias=ba1.bbd.px, local_file=../_bbgDB/Futures_gen/>
...updating security BA1 B:00_0_N Comdty
...loading local file <../_bbgDB/Futures_gen/ba1.bbd.px.pickle> for security=BA1 B:00_0_N Comdty
...updating timeseries for BA1 B:00_0_N Comdty --> loading data from 09/24/2019 to 10/01/2019
...saving BA1 B:00_0_N Comdty to local file <../_bbgDB/Futures_gen/ba1.bbd.px.pickle>

(210) <BBG SECURITY: tckr=PA1 B:00_0_D Comdty, alias=pa1.bbd, local_file=../_bbgDB/Futures_gen/>
...updating security PA1 B:00_0_D Comdty
...loading local file <../_bbgDB/Futures_gen/pa1.bbd.pickle> for security=PA1 B:00_0_D Comdty
...updating timeseries for PA1 B:00_0_D Comdty --> loading data from 09/24/2019 to 10/01/2019
...saving PA1 B:00_0_D Comdty to local file <../_bbgDB/Futures_gen/pa1.bbd.pickle>

(211) <BBG SECURITY: tckr=AD1 B:00_0_N Curncy, alias=ad1.bbd.px, local_file=../_bbgDB/F


(229) <BBG SECURITY: tckr=XBW1 B:00_0_N Comdty, alias=xbw1.bbd.px, local_file=../_bbgDB/Futures_gen/>
...updating security XBW1 B:00_0_N Comdty
...loading local file <../_bbgDB/Futures_gen/xbw1.bbd.px.pickle> for security=XBW1 B:00_0_N Comdty
...updating timeseries for XBW1 B:00_0_N Comdty --> loading data from 09/24/2019 to 10/01/2019
...saving XBW1 B:00_0_N Comdty to local file <../_bbgDB/Futures_gen/xbw1.bbd.px.pickle>
--------------UPDATING InterestRates----------------

(0) <BBG SECURITY: tckr=US0001M Index, alias=us.1m, local_file=../_bbgDB/InterestRates/>
...updating security US0001M Index
...loading local file <../_bbgDB/InterestRates/us.1m.pickle> for security=US0001M Index
...updating timeseries for US0001M Index --> loading data from 09/23/2019 to 10/01/2019
...saving US0001M Index to local file <../_bbgDB/InterestRates/us.1m.pickle>

(1) <BBG SECURITY: tckr=GDBR7 Index, alias=eu.7y, local_file=../_bbgDB/InterestRates/>
...updating security GDBR7 Index
...loading local file

...saving GSWISS10 Index to local file <../_bbgDB/InterestRates/ch.10y.pickle>

(22) <BBG SECURITY: tckr=USGG7YR Index, alias=us.7y, local_file=../_bbgDB/InterestRates/>
...updating security USGG7YR Index
...loading local file <../_bbgDB/InterestRates/us.7y.pickle> for security=USGG7YR Index
...updating timeseries for USGG7YR Index --> loading data from 09/24/2019 to 10/01/2019
...saving USGG7YR Index to local file <../_bbgDB/InterestRates/us.7y.pickle>

(23) <BBG SECURITY: tckr=GUKG10 Index Index, alias=uk.10y, local_file=../_bbgDB/InterestRates/>
...updating security GUKG10 Index Index
...loading local file <../_bbgDB/InterestRates/uk.10y.pickle> for security=GUKG10 Index Index
...updating timeseries for GUKG10 Index Index --> loading data from 09/24/2019 to 10/01/2019
...saving GUKG10 Index Index to local file <../_bbgDB/InterestRates/uk.10y.pickle>

(24) <BBG SECURITY: tckr=GDBR10 Index, alias=eu.10y, local_file=../_bbgDB/InterestRates/>
...updating security GDBR10 Index
...loading

...saving NOBRDEPA Index to local file <../_bbgDB/InterestRates/no.cb.pickle>

(43) <BBG SECURITY: tckr=CABROVER Index, alias=cn.cb, local_file=../_bbgDB/InterestRates/>
...updating security CABROVER Index
...loading local file <../_bbgDB/InterestRates/cn.cb.pickle> for security=CABROVER Index
...updating timeseries for CABROVER Index --> loading data from 09/24/2019 to 10/01/2019
...saving CABROVER Index to local file <../_bbgDB/InterestRates/cn.cb.pickle>

(44) <BBG SECURITY: tckr=SZLTHIGH Index, alias=ch.cb, local_file=../_bbgDB/InterestRates/>
...updating security SZLTHIGH Index
...loading local file <../_bbgDB/InterestRates/ch.cb.pickle> for security=SZLTHIGH Index
...updating timeseries for SZLTHIGH Index --> loading data from 06/21/2018 to 10/01/2019
...saving SZLTHIGH Index to local file <../_bbgDB/InterestRates/ch.cb.pickle>

(45) <BBG SECURITY: tckr=GNZGB2 Index, alias=nz.2y, local_file=../_bbgDB/InterestRates/>
...updating security GNZGB2 Index
...loading local file <../_bbg

In [11]:
ExpiredFutures.add_expired(expired['Futures'])
ExpiredFutures.save()

In [ ]:
# Database Health Files
databases = ['Futures', 'Futures_gen', 'FX', 'CoT', 'Index', 'InterestRates' ]
health = {}

for db in databases:
    health[db] = export_db_health(db)


ad.F10.pickle
ad.F18.pickle
ad.F19.pickle
ad.F20.pickle
ad.G18.pickle
ad.G19.pickle
ad.H00.pickle
ad.H01.pickle
ad.H02.pickle
ad.H03.pickle
ad.H04.pickle
ad.H05.pickle
ad.H06.pickle
ad.H07.pickle
ad.H08.pickle
ad.H09.pickle
ad.H10.pickle
ad.H11.pickle
ad.H12.pickle
ad.H13.pickle
ad.H14.pickle
ad.H15.pickle
ad.H16.pickle
ad.H17.pickle
ad.H18.pickle
ad.H19.pickle
ad.H20.pickle
ad.H21.pickle
ad.H22.pickle
ad.H23.pickle
ad.H24.pickle
ad.H87.pickle
ad.H88.pickle
ad.H89.pickle
ad.H90.pickle
ad.H91.pickle
ad.H92.pickle
ad.H93.pickle
ad.H94.pickle
ad.H95.pickle
ad.H96.pickle
ad.H97.pickle
ad.H98.pickle
ad.H99.pickle
ad.J17.pickle
ad.J18.pickle
ad.J19.pickle
ad.K17.pickle
ad.K18.pickle
ad.K19.pickle
ad.M00.pickle
ad.M01.pickle
ad.M02.pickle
ad.M03.pickle
ad.M04.pickle
ad.M05.pickle
ad.M06.pickle
ad.M07.pickle
ad.M08.pickle
ad.M09.pickle
ad.M10.pickle
ad.M11.pickle
ad.M12.pickle
ad.M13.pickle
ad.M14.pickle
ad.M15.pickle
ad.M16.pickle
ad.M17.pickle
ad.M18.pickle
ad.M19.pickle
ad.M20.pickle
ad.M21

bo.H92.pickle
bo.H93.pickle
bo.H94.pickle
bo.H95.pickle
bo.H96.pickle
bo.H97.pickle
bo.H98.pickle
bo.H99.pickle
bo.K00.pickle
bo.K01.pickle
bo.K02.pickle
bo.K03.pickle
bo.K04.pickle
bo.K05.pickle
bo.K06.pickle
bo.K07.pickle
bo.K08.pickle
bo.K09.pickle
bo.K10.pickle
bo.K11.pickle
bo.K12.pickle
bo.K13.pickle
bo.K14.pickle
bo.K15.pickle
bo.K16.pickle
bo.K17.pickle
bo.K18.pickle
bo.K19.pickle
bo.K20.pickle
bo.K21.pickle
bo.K60.pickle
bo.K61.pickle
bo.K62.pickle
bo.K63.pickle
bo.K64.pickle
bo.K65.pickle
bo.K66.pickle
bo.K67.pickle
bo.K68.pickle
bo.K69.pickle
bo.K70.pickle
bo.K71.pickle
bo.K72.pickle
bo.K73.pickle
bo.K74.pickle
bo.K75.pickle
bo.K76.pickle
bo.K77.pickle
bo.K78.pickle
bo.K79.pickle
bo.K80.pickle
bo.K81.pickle
bo.K82.pickle
bo.K83.pickle
bo.K84.pickle
bo.K85.pickle
bo.K86.pickle
bo.K87.pickle
bo.K88.pickle
bo.K89.pickle
bo.K90.pickle
bo.K91.pickle
bo.K92.pickle
bo.K93.pickle
bo.K94.pickle
bo.K95.pickle
bo.K96.pickle
bo.K97.pickle
bo.K98.pickle
bo.K99.pickle
bo.N00.pickle
bo.N01

c.H75.pickle
c.H76.pickle
c.H77.pickle
c.H78.pickle
c.H79.pickle
c.H80.pickle
c.H81.pickle
c.H82.pickle
c.H83.pickle
c.H84.pickle
c.H85.pickle
c.H86.pickle
c.H87.pickle
c.H88.pickle
c.H89.pickle
c.H90.pickle
c.H91.pickle
c.H92.pickle
c.H93.pickle
c.H94.pickle
c.H95.pickle
c.H96.pickle
c.H97.pickle
c.H98.pickle
c.H99.pickle
c.K00.pickle
c.K01.pickle
c.K02.pickle
c.K03.pickle
c.K04.pickle
c.K05.pickle
c.K06.pickle
c.K07.pickle
c.K08.pickle
c.K09.pickle
c.K10.pickle
c.K11.pickle
c.K12.pickle
c.K13.pickle
c.K14.pickle
c.K15.pickle
c.K16.pickle
c.K17.pickle
c.K18.pickle
c.K19.pickle
c.K20.pickle
c.K21.pickle
c.K60.pickle
c.K61.pickle
c.K62.pickle
c.K63.pickle
c.K64.pickle
c.K65.pickle
c.K66.pickle
c.K67.pickle
c.K68.pickle
c.K69.pickle
c.K70.pickle
c.K71.pickle
c.K72.pickle
c.K73.pickle
c.K74.pickle
c.K75.pickle
c.K76.pickle
c.K77.pickle
c.K78.pickle
c.K79.pickle
c.K80.pickle
c.K81.pickle
c.K82.pickle
c.K83.pickle
c.K84.pickle
c.K85.pickle
c.K86.pickle
c.K87.pickle
c.K88.pickle
c.K89.pickle

cd.H89.pickle
cd.H90.pickle
cd.H91.pickle
cd.H92.pickle
cd.H93.pickle
cd.H94.pickle
cd.H95.pickle
cd.H96.pickle
cd.H97.pickle
cd.H98.pickle
cd.H99.pickle
cd.J17.pickle
cd.J18.pickle
cd.J19.pickle
cd.K17.pickle
cd.K18.pickle
cd.K19.pickle
cd.M00.pickle
cd.M01.pickle
cd.M02.pickle
cd.M03.pickle
cd.M04.pickle
cd.M05.pickle
cd.M06.pickle
cd.M07.pickle
cd.M08.pickle
cd.M09.pickle
cd.M10.pickle
cd.M11.pickle
cd.M12.pickle
cd.M13.pickle
cd.M14.pickle
cd.M15.pickle
cd.M16.pickle
cd.M17.pickle
cd.M18.pickle
cd.M19.pickle
cd.M20.pickle
cd.M21.pickle
cd.M22.pickle
cd.M23.pickle
cd.M24.pickle
cd.M87.pickle
cd.M88.pickle
cd.M89.pickle
cd.M90.pickle
cd.M91.pickle
cd.M92.pickle
cd.M93.pickle
cd.M94.pickle
cd.M95.pickle
cd.M96.pickle
cd.M97.pickle
cd.M98.pickle
cd.M99.pickle
cd.N17.pickle
cd.N18.pickle
cd.N19.pickle
cd.Q17.pickle
cd.Q18.pickle
cd.Q19.pickle
cd.U00.pickle
cd.U01.pickle
cd.U02.pickle
cd.U03.pickle
cd.U04.pickle
cd.U05.pickle
cd.U06.pickle
cd.U07.pickle
cd.U08.pickle
cd.U09.pickle
cd.U10

cl.G19.pickle
cl.G20.pickle
cl.G21.pickle
cl.G22.pickle
cl.G23.pickle
cl.G24.pickle
cl.G25.pickle
cl.G26.pickle
cl.G27.pickle
cl.G28.pickle
cl.G29.pickle
cl.G30.pickle
cl.G84.pickle
cl.G85.pickle
cl.G86.pickle
cl.G87.pickle
cl.G88.pickle
cl.G89.pickle
cl.G90.pickle
cl.G91.pickle
cl.G92.pickle
cl.G93.pickle
cl.G94.pickle
cl.G95.pickle
cl.G96.pickle
cl.G97.pickle
cl.G98.pickle
cl.G99.pickle
cl.H00.pickle
cl.H01.pickle
cl.H02.pickle
cl.H03.pickle
cl.H04.pickle
cl.H05.pickle
cl.H06.pickle
cl.H07.pickle
cl.H08.pickle
cl.H09.pickle
cl.H10.pickle
cl.H11.pickle
cl.H12.pickle
cl.H13.pickle
cl.H14.pickle
cl.H15.pickle
cl.H16.pickle
cl.H17.pickle
cl.H18.pickle
cl.H19.pickle
cl.H20.pickle
cl.H21.pickle
cl.H22.pickle
cl.H23.pickle
cl.H24.pickle
cl.H25.pickle
cl.H26.pickle
cl.H27.pickle
cl.H28.pickle
cl.H29.pickle
cl.H84.pickle
cl.H85.pickle
cl.H86.pickle
cl.H87.pickle
cl.H88.pickle
cl.H89.pickle
cl.H90.pickle
cl.H91.pickle
cl.H92.pickle
cl.H93.pickle
cl.H94.pickle
cl.H95.pickle
cl.H96.pickle
cl.H97

cn.Z11.pickle
cn.Z12.pickle
cn.Z13.pickle
cn.Z14.pickle
cn.Z15.pickle
cn.Z16.pickle
cn.Z17.pickle
cn.Z18.pickle
cn.Z19.pickle
cn.Z89.pickle
cn.Z90.pickle
cn.Z91.pickle
cn.Z92.pickle
cn.Z93.pickle
cn.Z94.pickle
cn.Z95.pickle
cn.Z96.pickle
cn.Z97.pickle
cn.Z98.pickle
cn.Z99.pickle
co.F00.pickle
co.F01.pickle
co.F02.pickle
co.F03.pickle
co.F04.pickle
co.F05.pickle
co.F06.pickle
co.F07.pickle
co.F08.pickle
co.F09.pickle
co.F10.pickle
co.F11.pickle
co.F12.pickle
co.F13.pickle
co.F14.pickle
co.F15.pickle
co.F16.pickle
co.F17.pickle
co.F18.pickle
co.F19.pickle
co.F20.pickle
co.F21.pickle
co.F22.pickle
co.F23.pickle
co.F24.pickle
co.F25.pickle
co.F26.pickle
co.F27.pickle
co.F28.pickle
co.F89.pickle
co.F90.pickle
co.F91.pickle
co.F92.pickle
co.F93.pickle
co.F94.pickle
co.F95.pickle
co.F96.pickle
co.F97.pickle
co.F98.pickle
co.F99.pickle
co.G00.pickle
co.G01.pickle
co.G02.pickle
co.G03.pickle
co.G04.pickle
co.G05.pickle
co.G06.pickle
co.G07.pickle
co.G08.pickle
co.G09.pickle
co.G10.pickle
co.G11

ct.K71.pickle
ct.K72.pickle
ct.K73.pickle
ct.K74.pickle
ct.K75.pickle
ct.K76.pickle
ct.K77.pickle
ct.K78.pickle
ct.K79.pickle
ct.K80.pickle
ct.K81.pickle
ct.K82.pickle
ct.K83.pickle
ct.K84.pickle
ct.K85.pickle
ct.K86.pickle
ct.K87.pickle
ct.K88.pickle
ct.K89.pickle
ct.K90.pickle
ct.K91.pickle
ct.K92.pickle
ct.K93.pickle
ct.K94.pickle
ct.K95.pickle
ct.K96.pickle
ct.K97.pickle
ct.K98.pickle
ct.K99.pickle
ct.N00.pickle
ct.N01.pickle
ct.N02.pickle
ct.N03.pickle
ct.N04.pickle
ct.N05.pickle
ct.N06.pickle
ct.N07.pickle
ct.N08.pickle
ct.N09.pickle
ct.N10.pickle
ct.N11.pickle
ct.N12.pickle
ct.N13.pickle
ct.N14.pickle
ct.N15.pickle
ct.N16.pickle
ct.N17.pickle
ct.N18.pickle
ct.N19.pickle
ct.N20.pickle
ct.N21.pickle
ct.N22.pickle
ct.N60.pickle
ct.N61.pickle
ct.N62.pickle
ct.N63.pickle
ct.N64.pickle
ct.N65.pickle
ct.N66.pickle
ct.N67.pickle
ct.N68.pickle
ct.N69.pickle
ct.N70.pickle
ct.N71.pickle
ct.N72.pickle
ct.N73.pickle
ct.N74.pickle
ct.N75.pickle
ct.N76.pickle
ct.N77.pickle
ct.N78.pickle
ct.N79

ed.F16.pickle
ed.F17.pickle
ed.F18.pickle
ed.F19.pickle
ed.F20.pickle
ed.F96.pickle
ed.F97.pickle
ed.F98.pickle
ed.F99.pickle
ed.G00.pickle
ed.G01.pickle
ed.G02.pickle
ed.G03.pickle
ed.G04.pickle
ed.G05.pickle
ed.G06.pickle
ed.G07.pickle
ed.G08.pickle
ed.G09.pickle
ed.G10.pickle
ed.G11.pickle
ed.G12.pickle
ed.G13.pickle
ed.G14.pickle
ed.G15.pickle
ed.G16.pickle
ed.G17.pickle
ed.G18.pickle
ed.G19.pickle
ed.G20.pickle
ed.G96.pickle
ed.G97.pickle
ed.G98.pickle
ed.G99.pickle
ed.H00.pickle
ed.H01.pickle
ed.H02.pickle
ed.H03.pickle
ed.H04.pickle
ed.H05.pickle
ed.H06.pickle
ed.H07.pickle
ed.H08.pickle
ed.H09.pickle
ed.H10.pickle
ed.H11.pickle
ed.H12.pickle
ed.H13.pickle
ed.H14.pickle
ed.H15.pickle
ed.H16.pickle
ed.H17.pickle
ed.H18.pickle
ed.H19.pickle
ed.H20.pickle
ed.H21.pickle
ed.H22.pickle
ed.H23.pickle
ed.H24.pickle
ed.H25.pickle
ed.H26.pickle
ed.H27.pickle
ed.H28.pickle
ed.H29.pickle
ed.H87.pickle
ed.H88.pickle
ed.H89.pickle
ed.H90.pickle
ed.H91.pickle
ed.H92.pickle
ed.H93.pickle
ed.H94

eo.U02.pickle
eo.U03.pickle
eo.U04.pickle
eo.U05.pickle
eo.U06.pickle
eo.U07.pickle
eo.U08.pickle
eo.U09.pickle
eo.U10.pickle
eo.U11.pickle
eo.U12.pickle
eo.U13.pickle
eo.U14.pickle
eo.U15.pickle
eo.U16.pickle
eo.U17.pickle
eo.U18.pickle
eo.U19.pickle
eo.U20.pickle
eo.U89.pickle
eo.U90.pickle
eo.U91.pickle
eo.U92.pickle
eo.U93.pickle
eo.U94.pickle
eo.U95.pickle
eo.U96.pickle
eo.U97.pickle
eo.U98.pickle
eo.U99.pickle
eo.V00.pickle
eo.V01.pickle
eo.V02.pickle
eo.V03.pickle
eo.V04.pickle
eo.V05.pickle
eo.V06.pickle
eo.V07.pickle
eo.V08.pickle
eo.V09.pickle
eo.V10.pickle
eo.V11.pickle
eo.V12.pickle
eo.V13.pickle
eo.V14.pickle
eo.V15.pickle
eo.V16.pickle
eo.V17.pickle
eo.V18.pickle
eo.V19.pickle
eo.V89.pickle
eo.V90.pickle
eo.V91.pickle
eo.V92.pickle
eo.V93.pickle
eo.V94.pickle
eo.V95.pickle
eo.V96.pickle
eo.V97.pickle
eo.V98.pickle
eo.V99.pickle
eo.X00.pickle
eo.X01.pickle
eo.X02.pickle
eo.X03.pickle
eo.X04.pickle
eo.X05.pickle
eo.X06.pickle
eo.X07.pickle
eo.X08.pickle
eo.X09.pickle
eo.X10

In [13]:
# Zip Database into single file for export
fldr = '../_bbgDB'
zipfldr = '../_out/'
zipfilename = 'bbgDB-' + datetime.now().strftime('%Y-%m-%d') + '.zip'

zip_folder(fldr, zipfldr, zipfilename)